In [2]:
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

## 5.4.1 곱셈 계층

In [5]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = self.x * self.y

        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

In [7]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [9]:
dprice = 1

dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple_price, dtax)
print(dapple, dapple_num)

1.1 200
2.2 110.00000000000001


## 5.4.2 덧셈 계층

In [48]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        return x + y
    
    def backward(self, dout):
        return dout, dout

In [50]:
#그림 5-17 forward
apple = 100
orange = 150
apple_num = 2
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_price_layer = AddLayer()
mul_tax_layer = MulLayer()

apple_all_price = mul_apple_layer.forward(apple, apple_num)
orange_all_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_price_layer.forward(apple_all_price, orange_all_price)
price_with_tax = mul_tax_layer.forward(all_price, tax)

print(apple_all_price, orange_all_price, all_price, price_with_tax)

200 450 650 715.0000000000001


In [12]:
#그림 5-17 backward

dprice = 1
dall_price, dtax = mul_tax_layer.backward(1)
dapple_all_price, dorange_all_price = add_price_layer.backward(dall_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_all_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_all_price)

print(dprice, dall_price, dapple_all_price, dorange_all_price, dapple_num, dapple, dorange, dorange_num, dtax)

1 1.1 1.1 1.1 110.00000000000001 2.2 3.3000000000000003 165.0 650


## 5.5 활성화 함수 계층 구현하기
### 5.5.1 ReLU 계층

In [13]:
class ReLU:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [42]:
relu_layer = ReLU

print(relu_layer.forward(relu_layer, np.array([5])))
print(relu_layer.backward(relu_layer, np.array([13])))

[5]
[13]


### 5.5.2 Sigmoid 계층

In [37]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out
    
    def backward(self, dout):
        return dout * self.out * (1 - self.out)

In [46]:
sigmoid_layer = Sigmoid

print(sigmoid_layer.forward(sigmoid_layer, 5))
print(sigmoid_layer.backward(sigmoid_layer, 1))

0.9933071490757153
0.006648056670790033


## 5.6 Affine/Softmax 계층 구현하기
### 5.6.1 Affine 계층

In [51]:
X = np.random.rand(2) # (1, 2)
W = np.random.rand(2, 3) # (2, 3)
b = np.random.rand(3) # (1, 3)

print(X.shape, W.shape, b.shape)

y = np.dot(X, W) + b

print(y.shape) # (1, 3)

(2,) (2, 3) (3,)
(3,)


In [52]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [53]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
print(dY)
dB = np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


### 2차원용 Affine

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.b = np.sum(dout, axis=0)

        return dx

### 5.6.3 Softmax-with-Loss 계층